In [41]:
import glob
import numpy as np

import matplotlib.pyplot as plt

In [1]:
from pathlib import Path
import argparse
import cv2
import matplotlib.cm as cm
import torch

from SuperGluePretrainedNetwork.models.matching import Matching
from SuperGluePretrainedNetwork.models.utils import (AverageTimer, VideoStreamer,
                          make_matching_plot_fast, frame2tensor)

## 1. Load SuperPoint and SuperGlue models

In [9]:
force_cpu = False
device = 'cuda' if torch.cuda.is_available() and not force_cpu else 'cpu'
config = {
        'superpoint': {
            'nms_radius': 4,
            'keypoint_threshold': 0.005,
            'max_keypoints': -1
        },
        'superglue': {
            'weights': 'indoor',
            'sinkhorn_iterations': 20,
            'match_threshold': 0.2,
        }
    }

In [11]:
matching = Matching(config).eval().to(device)
keys = ['keypoints', 'scores', 'descriptors']

Loaded SuperPoint model
Loaded SuperGlue model ("indoor" weights)


In [4]:
all_scans = glob.glob("../RGBD/garden*")
images_set = []
print("Found", len(all_scans), "scans.")
for scan_folder in all_scans:
    scan_images = glob.glob(scan_folder +"/*_r.jpg")
    scan_images.sort()
    images_set = images_set + scan_images
print("Found", len(images_set), "images.")

Found 133 scans.
Found 1197 images.


In [14]:
test_images = glob.glob("test_images/*.png")
test_images.sort()
print("Found", len(test_images), "test images")

Found 9 test images


In [29]:
test_images[idx]

'test_images/122.png'

In [25]:
idx = 0
frame = cv2.imread(test_images[idx])
frame = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
frame_tensor = frame2tensor(frame, device)


last_data = matching.superpoint({'image': frame_tensor})
last_data = {k+'0': last_data[k] for k in keys}
last_data['image0'] = frame_tensor
last_frame = frame
last_image_id = 0

In [31]:
frame = cv2.imread("9_r.jpg")
frame = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)

frame_tensor = frame2tensor(frame, device)
pred = matching({**last_data, 'image1': frame_tensor})
kpts0 = last_data['keypoints0'][0].cpu().numpy()
kpts1 = pred['keypoints1'][0].cpu().numpy()
matches = pred['matches0'][0].cpu().numpy()
confidence = pred['matching_scores0'][0].cpu().detach().numpy()

In [39]:
stem0 = 0
stem1 = 1
valid = matches > -1
mkpts0 = kpts0[valid]
mkpts1 = kpts1[matches[valid]]
color = cm.jet(confidence[valid])
text = [
        'SuperGlue',
        'Keypoints: {}:{}'.format(len(kpts0), len(kpts1)),
        'Matches: {}'.format(len(mkpts0))
    ]
k_thresh = matching.superpoint.config['keypoint_threshold']
m_thresh = matching.superglue.config['match_threshold']
small_text = [
        'Keypoint Threshold: {:.4f}'.format(k_thresh),
        'Match Threshold: {:.2f}'.format(m_thresh),
        'Image Pair: {:06}:{:06}'.format(stem0, stem1),
    ]
out = make_matching_plot_fast(
            last_frame, frame, kpts0, kpts1, mkpts0, mkpts1, color, text,
            path=None, show_keypoints=False, small_text=small_text)


In [48]:
cv2.imwrite("out.jpg",out)

True